In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import math as ma
from astropy.io import fits
from astropy.time import Time
from matplotlib import pyplot as plt
from matplotlib.axis import Axis
from pyuvdata import UVData
import matplotlib as mpl

In [2]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/thermal_only/3_5_one_side/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
    #with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])

    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1]*data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0]*data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp["time_index"].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp["time_index"].map(lambda t: int(time_blocks[t]))

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop=True)

In [ ]:
df_sel_rfi_fast = df_sel_rfi_fast[df_sel_rfi_fast['obs_id'] == 1095451432].reset_index(drop=True)
df_sel_rfi_fast[((df_sel_rfi_fast["XX"] < -5) | (df_sel_rfi_fast["XX"] > 5)) | ((df_sel_rfi_fast["YY"] < -5) | (df_sel_rfi_fast["YY"] > 5))]

In [10]:
# Selected outliers for thermal noise only based on winsorizing z_score

df_sel_rfi_fast[((df_sel_rfi_fast["XX"] < -5)) | ((df_sel_rfi_fast["YY"] < -5))].reset_index(drop=True)

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
0,-5.047529,-1.623106,0.917379,0.810451,1095450704,5,564,124,5
1,-5.011579,-2.345671,2.548353,0.125190,1095450704,10,161,33,10
2,-5.336892,-0.215848,-0.265656,0.107893,1095450824,23,132,34,9
3,0.853553,0.482752,0.325399,-5.287469,1095451432,89,39,30,5
4,-5.080586,0.095828,-0.086522,0.951310,1095451552,104,282,38,6
5,1.580151,1.682172,-1.926710,-5.288006,1095451800,135,610,38,9


In [15]:
14*16*768*128

22020096

In [11]:
df_sel_rfi_fast['obs_id'].value_counts()

obs_id
1095450704    1376256
1095450824    1376256
1095450944    1376256
1095451064    1376256
1095451192    1376256
1095451312    1376256
1095451432    1376256
1095451552    1376256
1095451680    1376256
1095451800    1376256
1095451920    1376256
1095452040    1376256
1095452168    1376256
1095452288    1376256
1095452408    1376256
1095452528    1376256
Name: count, dtype: int64

---

#### Outliers location

In [7]:
df_out_loc = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/thermal_only/3_5_one_side/outliers_location_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
    #with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/outliers_location_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        outliers_mask = f["outliers_mask"][:]
        obs_id = f["obs_id"][:].astype(str)
        time_blocks = f["time_blocks"][:]

    flat_data = outliers_mask.reshape(-1, outliers_mask.shape[-1])

    df_outliers = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_outliers["obs_id"] = np.tile(obs_id, outliers_mask.shape[1]*outliers_mask.shape[2])
    df_outliers["time_index"] = np.repeat(np.arange(outliers_mask.shape[0]), outliers_mask.shape[1] * outliers_mask.shape[2])
    df_outliers["frequency"] = np.tile(np.arange(outliers_mask.shape[2]), outliers_mask.shape[0] * outliers_mask.shape[1])
    df_outliers["antenna"] = np.tile(np.repeat(np.arange(outliers_mask.shape[1]), outliers_mask.shape[2]), outliers_mask.shape[0])
    df_outliers["obs_id"] = df_outliers['time_index'].map(lambda t: int(obs_id[t]))
    df_outliers["timeblock"] = df_outliers['time_index'].map(lambda t: int(time_blocks[t]))

    df_out_loc = pd.concat([df_out_loc, df_outliers]).reset_index(drop=True)

In [ ]:
# Selected outliers for thermal noise only

df_out_loc[df_out_loc[["XX", "YY"]].any(axis=1)].reset_index(drop=True)

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
0,True,False,False,False,1095450704,5,564,124,5
1,True,False,False,False,1095450704,10,161,33,10
2,True,False,False,False,1095450824,23,132,34,9
3,False,False,False,True,1095451432,89,39,30,5
4,True,False,False,False,1095451552,104,282,38,6
5,False,False,False,True,1095451800,135,610,38,9


In [ ]:
sel_df_outliers = df_out_loc[df_out_loc['obs_id'] == 1095451432]['frequency'].value_counts().to_frame().reset_index()
sel_df_outliers = sel_df_outliers[sel_df_outliers['count'] > 3]['frequency'].to_list()

In [ ]:
df_out_loc_sel = df_out_loc[df_out_loc["obs_id"] == 1095451432].reset_index(drop=True)
df_out_loc_sel = df_out_loc_sel[df_out_loc_sel[["XX", "YY"]].any(axis=1)].reset_index(drop=True)

In [ ]:
df_out_loc_sel

In [ ]:
df_out_loc_sel[["XX", "YY"]].isna()